**Modeling Water Movement and Reservoir Operations with *mosartwmpy***
<br>
CSDMS 2022: Environmental Extremes and Earthscape Evolution (E4)
<br>
18 May 2022

Travis Thurber \<travis.thurber@pnnl.gov\>
<br>
Chris Vernon \<chris.vernon@pnnl.gov\>
<br>
Pacific Northwest National Laboratory (PNNL)

<br>
<br>

**Materials**

* Workshop repository: https://github.com/IMMM-SFA/csdms-mosartwmpy-clinic-2022
* `mosartwmpy` docs: https://mosartwmpy.readthedocs.io/en/latest/

<br>
<br>

**Goals**

* Learn the basics of configuring and running the `mosartwmpy` model
* Learn how to Visualize `mosartwmpy` output data
* Study the historical drought of the early 21st-century (1998 - 2014)
* Compare the water deficit during this drought with the US Drought Monitor figure:

<img src="drought_20120925_usdm.png" alt="USDM Sep 2012" width=720>


<br>
<br>

**Acknowledgements**

* [**MSD-LIVE**](https://msdlive.org/) is building a collaborative data and computational platform for the MultiSector Dynamics community. They have provided today's Jupyter environment.
* [**IM3**](https://im3.pnnl.gov) develops innovative modeling to explore how human and natural landscapes in the USA co-evolve in response to short-term shocks and long-term influences. They currently fund `mosartwmpy` development.
* [**CSDMS**](https://csdms.colorado.edu/wiki/Main_Page) supports the Earth systems modeling community by disseminating integrated software modules. They host today's clinic 🤓.
<br>
<br>

***mosartwmpy***

[`mosartwmpy`](https://mosartwmpy.readthedocs.io/en/latest/) is a large-scale water routing and reservoir management model written in Python, translated from the FORTRAN version known as MOSART-WM (which stands for Model Of Scale Adaptive River Transport and Water Management). `mosartwmpy` implements the Basic Model Interface (BMI), a common set of methods for controlling a model that is likely familiar to members of the CSDMS community. For configuration, `mosartwmpy` reads a single YAML file that extends sensible defaults. The typical grid cell resolution is 1/8 degree across the conterminous United States, with a temporal output resolution of 3 hour timesteps. Output includes water levels and flow at each grid cell and in each reservoir, as well as water supply and deficit at each grid cell.

`mosartwmpy` is best used to study fundamental science questions inspecting general trends of how the water system may evolve into the future under different scenarios, and is not meant to replace high resolution applied models.

Active research topics involving the model includes:
* drought and water deficit
* reservoir release policies
* farmer crop choices
* hydropower
<br>
<br>

**Input**

`mosartwmpy` expects gridded input data--and provides gridded output data--in the WGS84 (EPSG 4326) projection. Data is typically provided and generated at 1/8 degree spatial resolution, though other resolutions can work if provided consistently throughout the input files.

Several input datasets are required:

| Name | Description | Sample File |
| ---- | ----------- | ----------- |
| Domain and river network parameters | NetCDF file that specifies the flow direction, land and channel properties, and coordinates of each grid cell | mosartwmpy_river_parameters.nc |
| Runoff timeseries | NetCDF file(s) that specifies the surface and subsurface runoff at each grid cell over time in mm/s | mosartwmpy_runoff_1980_2019.nc |
| Demand timeseries | NetCDF file(s) that specifies the consumptive water demand at each grid cell over time in m<sup>3</sup>/s | mosartwmpy_demand_1980_2019.nc |
| Dam/reservoir parameters | NetCDF file that specifies the dam/reservoir properties and grid cell location | mosartwmpy_reservoir_parameters.nc |
| Dam monthly mean flow | Parquet file that specifies the long term naturalized mean flow by month at each dam | mosartwmpy_dam_mean_monthly_flow.parquet |
| Reservoir monthly mean demand | Parquet file that specifies the long term mean demand by month for the water in each reservoir | mosartwmpy_reservoir_mean_monthly_demand.parquet |
| Reservoir dependency database | Parquet file that specifies which reservoirs each grid cell can consume water from | mosartwmpy_reservoir_dependency_database.parquet |

<br>
<br>

**Integrations**

`mosartwmpy` can integrate with other models that provide surface/subsurface runoff or water demand. Historically this has often meant [VIC](https://vic.readthedocs.io/en/master/) or [CLM](https://www.cesm.ucar.edu/models/clm/) for runoff, and [GCAM](http://www.globalchange.umd.edu/gcam/) for water demand.

<br>

**About the Model**

Simplified flow diagram:

<img src="wmpy_flow.png" alt="mosartwmpy flow diagram" width=720>

<br>

Geospatial view (web mercator) of the grid cells relating to Hoover Dam and the Lower Colorado:
<br>
> Red dot: Hoover Dam / Lake Mead grid cell
> <br>
> Blue highlight: Grid cell path of river channel leaving the Hoover Dam
> <br>
> Green highlight: Grid cells allowed to consume water from Lake Mead
> <br>
> Gray border: Other grid cells in the Lower Colorado Basin

<img src="hoover.png" alt="hoover domain" width=720>

<br>
<br>

**Installation**

For this clinic, `mosartwmpy` is already installed into the Jupyter environment so you don't have to worry about it.

But for reference, there are three ways to install `mosartwmpy`:

* pip: `pip install mosartmpwy`
* conda: `conda install -c conda-forge mosartwmpy`
* docker (experimental): `docker pull thurber/mosartwmpy:latest`
* GitHub: https://github.com/IMMM-SFA/mosartwmpy

<br>
<br>

**Configuration**

`mosartwmpy` model runs are defined by the merger of the [default configuration](https://github.com/IMMM-SFA/mosartwmpy/blob/main/mosartwmpy/config_defaults.yaml) and the user configuration YAML file. This means you only need to specify configuration options that you want to change from their defaults--typically just a name and dates for the desired simulation period and the paths to the input files.

For instance, a basic user configuration file might look like this:

<br>

`my_config.yaml`
```yaml
simulation:
    name: My Simulation
    output_path: .
    restart_file: ~
    start_date: 1980-01-01
    end_date: 1980-01-31

grid:
  path: /home/data/mosartwmpy_river_parameters.nc

runoff:
  path: /home/data/mosartwmpy_runoff_1980_2019.nc

water_management:
  enabled: true
  demand:
    path: /home/data/mosartwmpy_demand_1980_2019.nc
  reservoirs:
    parameters:
      path: /home/data/mosartwmpy_reservoir_parameters.nc
    dependencies:
      path: /home/data/mosartwmpy_reservoir_dependency_database.parquet
    streamflow:
      path: /home/data/mosartwmpy_dam_mean_monthly_flow.parquet
    demand:
      path: /home/data/mosartwmpy_reservoir_mean_monthly_demand.parquet
```

<br>

<mark style="padding: 2px 3px;">**Let's create a configuration file together to use for this clinic!**</mark>


<br>
<br>

**Running the Model**

<mark style="padding: 2px 3px;">**Let's practice running the `mosartwmpy` model together!**</mark>


In [ ]:
from mosartwmpy import Model

# Our code will go here!

# 1) Create a Model instance
model = Model()

# 2) Initialize the Model instance using our configuration file
model.initialize('CSDMS_config.yaml')

# 3) Run for a single timestep
model.update()

# 4) Run until the end of the first month
model.update_until(model.get_end_time())


<br>

<mark style="padding: 2px 3px;">**Let's learn about restart files!**</mark>

In [ ]:
# 1) Update the configuration file to include a restart file

# 2) Update the start and end dates to reflect the time period implied by the restart file

# 3) Create, initialize, and run a new Model instance!
model_restart = Model()
model_restart.initialize('CSDMS_config_with_restart.yaml')
model_restart.update_until()

<br>
<br>

**Processing Output**

<mark style="padding: 2px 3px;">**Let's visualize `mosartwmpy` output!**</mark>


In [ ]:
# 1) get a list of model output variables using the BMI's `get_output_var_names()` method
model.get_ouptput_var_names()


In [ ]:
# 2) get the flattened numpy.ndarray of values for an output variable using the BMI's `get_value_ptr()` method
model.get_value_ptr('surface_water_amount')


In [ ]:
# 3) plot the instantaneous values of an output variable using the Model's `plot_variable()` method
model.plot_variable('surface_water_amount', log_scale=True)


In [ ]:
from mosartwmpy.plotting.plot import plot_reservoir, plot_variable


In [ ]:
# 4) plot a reservoir's behavior using mosartwmpy's utility method
plot_reservoir(
    model=model,
    grand_id=310,
    start='1981-01-01',
    end='1981-01-31',
)


In [ ]:
# 5) create an interactive timeseries plot of an output variable using mosartwmpy's utility method
plot_variable(
    model=model,
    variable='channel_outflow',
    start='1981-01-01',
    end='1981-01-31',
    log_scale=True,
    cmap='autumn_r',
)


<br>
<br>

**Visualizing the Historical 21st-Century Drought**

<mark style="padding: 2px 3px;">**Let's use the provided simulation output to recreate the Drought Monitor visualization**</mark>

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
# wildcard path to the simulation output files
simulation_output_glob = '/home/data/mosartwmpy_output/*.nc'

# variable name of the water deficit in the output files
water_deficit_variable = 'WRM_DEFICIT'

# variable name of the time dimension
time_variable = 'time'


In [ ]:
# open the data using xarray
simulation_output = xr.open_mfdataset(simulation_output_glob)


In [ ]:
# calculate the mean and standard deviation of the water deficit over the entire simulation
deficit_mean = simulation_output[water_deficit_variable].mean(dim=time_variable).compute()
deficit_std = simulation_output[water_deficit_variable].std(dim=time_variable).compute()


In [ ]:
# for simplicity, resample the deficit to the yearly mean
# note that this resampling somewhat obfuscates just how intense the drought was in the summer months!
deficit_by_year = simulation_output[[water_deficit_variable]].resample({ time_variable: '1y' }).mean(time_variable).compute()


In [ ]:
# let's define the deficit intensity as standard deviations worse than the mean
deficit_intensity = ((deficit_by_year[water_deficit_variable] - deficit_mean) / deficit_std).clip(0)


In [ ]:
# a method to plot the deficit intensity over multiple years
def plot_deficit_intensity(
    deficit_intensity,
    start_time,
    end_time,
    time_variable='time',
    buckets=[0, 0.2, 0.4, 0.6, 0.8, 1],
    colors=['none', '#fffc54', '#f5d38a', '#e49724', '#d42c1f', '#69110a'],
    figsize=(20,20),
    col_wrap=5,
    include_month=False,
):
    
    subset = deficit_intensity.sel({ time_variable: slice(start_time, end_time) })
    
    facets = subset.plot(
        colors=colors,
        levels=buckets,
        col=time_variable,
        col_wrap=col_wrap,
        cbar_kwargs={'label': 'Standard deviations worse than the mean deficit', 'aspect': 40, 'orientation': 'horizontal', 'location': 'top'},
        subplot_kws={'projection': ccrs.PlateCarree()},
        figsize=figsize
    )
    
    for i in range(len(subset[time_variable])):
        
        t = subset.isel({ time_variable: i })[time_variable].dt.date.values[()]
        
        ax = facets.axes.flatten()[i]
        
        ax.set_title(f'{t.year}' + (f'-{t.month}' if include_month else ''))
        
        ax.add_feature(cfeature.NaturalEarthFeature(
            'physical', 'coastline', scale='50m',
            edgecolor='black', facecolor='none', alpha=0.75))
        
        ax.add_feature(cfeature.NaturalEarthFeature(
            'cultural', 'admin_0_boundary_lines_land', scale='50m',
            edgecolor='gray', facecolor='none', alpha=0.75))
        
        ax.add_feature(cfeature.NaturalEarthFeature(
            'cultural', 'admin_1_states_provinces_lines', scale='50m',
            edgecolor='gray', facecolor='none', alpha=0.75))
        
        ax.add_feature(cfeature.NaturalEarthFeature(
            'physical', 'lakes', scale='50m',
            edgecolor='lightblue', facecolor='lightblue', alpha=0.5))
        
        ax.add_feature(cfeature.NaturalEarthFeature(
            'physical', 'rivers_lake_centerlines', scale='50m',
            edgecolor='lightblue', facecolor='none', alpha=0.75))
        
        # if t.year == 2012:
        #     for spine in ax.spines.values():
        #         spine.set_edgecolor('red')
        
    facets.fig.suptitle(f'Deficit Intensity', fontsize=28)


In [ ]:
# let's see how it looks!
plot_deficit_intensity(deficit_intensity, '1995', '2019')


In [ ]:
# Let's look at the monthtly breakdown for 2012:
deficit_2012 = simulation_output[[water_deficit_variable]].sel({ time_variable: '2012' }).resample({ time_variable: '1M' }).mean(time_variable).compute()
intensity_2012 = ((deficit_2012[water_deficit_variable] - deficit_mean) / deficit_std).clip(0)
plot_deficit_intensity(intensity_2012, '2012-01', '2012-12', buckets=[0, 0.5, 1, 1.5, 2, 2.5], figsize=(20,10), col_wrap=6, include_month=True)

Here's the drought monitor image again for reference:

<img src="drought_20120925_usdm.png" alt="USDM Sep 2012" width=720>


<br>
<br>